# Instalando bibliotecas

In [2]:
!pip install boto3

     ---------------------------------------- 0.0/139.9 kB ? eta -:--:--
     -------------------------------------- 139.9/139.9 kB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/84.8 kB ? eta -:--:--
     ---------------------------------------- 84.8/84.8 kB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/13.5 MB ? eta -:--:--
     -- ------------------------------------- 0.7/13.5 MB 23.0 MB/s eta 0:00:01
     ------- -------------------------------- 2.5/13.5 MB 32.6 MB/s eta 0:00:01
     ---------------- ----------------------- 5.4/13.5 MB 43.3 MB/s eta 0:00:01
     ----------------------------- --------- 10.3/13.5 MB 59.5 MB/s eta 0:00:01
     -------------------------------------  13.5/13.5 MB 108.8 MB/s eta 0:00:01
     -------------------------------------  13.5/13.5 MB 108.8 MB/s eta 0:00:01
     --------------------------------------- 13.5/13.5 MB 59.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Importando bibliotecas

In [ ]:
import boto3
import base64

### Conectando aos serviços AWS Rekognition e Textract

Para usar os serviços de Visão Computacional da AWS em Python, utilizamos a biblioteca `boto3`, que é o SDK oficial da AWS.


In [28]:
rekognition = boto3.client('rekognition', region_name='us-east-1')
textract = boto3.client('textract', region_name='us-east-1') 

### Função: `decode_base64_image`

Esta função tem como objetivo decodificar uma imagem que está em formato Base64, retornando os bytes da imagem.



In [32]:
def decode_base64_image(base64_string):
    return base64.b64decode(base64_string.split(",")[-1])

### Função: `compare_faces`

Esta função utiliza o serviço **Amazon Rekognition** para comparar duas imagens de rosto: uma selfie e uma foto presente em um documento (por exemplo, RG ou CNH).


In [33]:
def compare_faces(selfie_bytes, doc_bytes, similarity_threshold=90):
    response = rekognition.compare_faces(
        SourceImage={'Bytes': selfie_bytes},
        TargetImage={'Bytes': doc_bytes},
        SimilarityThreshold=similarity_threshold
    )
    for faceMatch in response['FaceMatches']:
        similarity = faceMatch['Similarity']
        return similarity
    return 0  

### Função: `extract_text_from_image`

Esta função utiliza o serviço **Amazon Rekognition** para extrair texto de uma imagem. Ela retorna todas as linhas de texto detectadas pela ferramenta.


In [ ]:
def extract_text_from_image(image_bytes):
    response = rekognition.detect_text(Image={'Bytes': image_bytes})
    detected_texts = [text['DetectedText'] for text in response['TextDetections'] if text['Type'] == 'LINE']
    return detected_texts

### Função: `extract_info_from_text`

Esta função extrai informações específicas (como nome e CPF) de uma lista de linhas de texto. Ela busca palavras-chave e tenta associar os valores encontrados às informações desejadas.


In [ ]:
def extract_info_from_text(text_lines):
    info = {}
    campos = {
        "nome": ["NOME"],
        "cpf": ["CPF"]
    }

    for i, line in enumerate(text_lines):
        for chave, palavras in campos.items():
            if any(palavra in line.upper() for palavra in palavras):
                # Tenta pegar a próxima linha como valor
                if i + 1 < len(text_lines):
                    valor = text_lines[i + 1].strip()
                    if chave not in info:  # Para evitar sobrescrita
                        info[chave] = valor
    return info


### Função: `processar_documento`

Esta função realiza o processamento completo de dois documentos: uma **selfie** e uma **CNH** (Carteira Nacional de Habilitação), para verificar a similaridade entre os rostos e extrair informações relevantes da CNH.


In [34]:
def processar_documento(selfie_base64, cnh_base64):
    selfie_bytes = decode_base64_image(selfie_base64)
    cnh_bytes = decode_base64_image(cnh_base64)

    # 1. Comparar rosto
    similaridade = compare_faces(selfie_bytes, cnh_bytes)
    print(f"Similaridade: {similaridade:.2f}%")

    if similaridade < 90:
        return similaridade

    # 2. Extrair texto
    textos = extract_text_from_image(cnh_bytes)
    info = extract_info_from_text(textos)
    
    return {
        "similaridade": similaridade,
        "informacoes_extraidas": info
    }

### Função: `extrair_endereco_por_nome`

Esta função extrai o endereço associado a um nome específico de uma lista de linhas de texto. A função procura pelo nome desejado e, se encontrado, tenta retornar o endereço que está na linha 10 após o nome.


In [ ]:
    
def extrair_endereco_por_nome(text_lines, nome_alvo):
    for i, linha in enumerate(text_lines):
        if nome_alvo.upper() in linha.upper():
            if len(text_lines) > 10:
                return text_lines[10].strip()
            else:
                return None
    return None

### Função: `processar_conta_por_nome`

Esta função processa uma imagem de conta (ou outro documento) e busca um **endereço** associado a um **nome específico**. Ela extrai o texto da imagem e, se o nome for encontrado, tenta extrair o endereço relacionado.


In [ ]:
def processar_conta_por_nome(caminho_arquivo, nome_alvo, mostrar_tudo=False):
    with open(caminho_arquivo, "rb") as f:
        imagem_bytes = f.read()
    linhas = extract_text_from_image(imagem_bytes)

    endereco = extrair_endereco_por_nome(linhas, nome_alvo)
    return {
        "nome_encontrado": nome_alvo if endereco else None,
        "endereco": endereco
    }

In [37]:
import base64

def carregar_imagem_base64(caminho):
    with open(caminho, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

# Caminhos locais das imagens
caminho_selfie = "self.jpeg"
caminho_cnh = "cnh.jpeg"

# Converte para base64
selfie_base64 = carregar_imagem_base64(caminho_selfie)
cnh_base64 = carregar_imagem_base64(caminho_cnh)

# Chama a função principal
resultado_imilaridade = processar_documento(selfie_base64, cnh_base64)

if resultado_imilaridade['similaridade'] > 99.:
    print("Resto são iguais, validando conta")
    resultado = processar_conta_por_nome("conta.jpeg", resultado_imilaridade['informacoes_extraidas']['nome'], mostrar_tudo=True)
    print("\n=== Resultado extraído ===")
    print(resultado)

Similaridade: 99.99%
Resto são iguais, validando conta

=== Resultado extraído ===
{'nome_encontrado': 'IGOR SILVA ALVES', 'endereco': 'R EDGAR AQUINO DUARTE, 123'}
